This is a start I will clean it up
TO DO

1. Load all the modules
2. read the training pool 
3. select random training set of 25 structure from the pool

In [9]:
import os
from ase.io import read, write
zundel_pool = read('/Users/shubhamdongriyal/my-drive/SabIA/ConferencesAndTalks/Brazil_july2024/tutorial/AL-tutorial-main/initial-datasets/zundel/train.extxyz', index=':')

In [2]:
import warnings
warnings.filterwarnings("ignore")
from mace.cli.run_train import main as mace_run_train_main
import sys
import logging

def train_mace(config_file_path):
    logging.getLogger().handlers.clear()
    sys.argv = ["program", "--config", config_file_path]
    mace_run_train_main()

In [3]:
print(f'Total number of structures in pool {len(zundel_pool)}')
import numpy as np

np.random.seed(0)
selected_indices = np.random.choice(len(zundel_pool), size=10, replace=False)
print(f'Selected indices: {selected_indices}')
initial_training_set = [zundel_pool[i] for i in selected_indices]

# Save the selected structures to a new file
write('initial_training_set.extxyz', initial_training_set, format='extxyz')


Total number of structures in pool 10001
Selected indices: [9395  898 2398 4737 2343 5111 5507 6452 2670 3188]


Hyperparameters for the committee members

In [4]:
# os.mkdir('config)
%%writefile config/config-01.yml

model: "MACE"
num_channels: 32
max_L: 0
r_max: 4.0
name: "mace_com1"
model_dir: "MACE_models"
log_dir: "MACE_models"
checkpoints_dir: "MACE_models"
results_dir: "MACE_models"
train_file: "initial_training_set.extxyz"
energy_key: "REF_energy"
forces_key: "REF_forces"
device: cpu
batch_size: 10
max_num_epochs: 20
swa: True
seed: 1

UsageError: Line magic function `%%writefile` not found.


In [5]:
%%writefile config/config-02.yml

model: "MACE"
num_channels: 32
max_L: 0
r_max: 4.0
name: "mace_com2"
model_dir: "MACE_models"
log_dir: "MACE_models"
checkpoints_dir: "MACE_models"
results_dir: "MACE_models"
train_file: "initial_training_set.extxyz"
energy_key: "REF_energy"
forces_key: "REF_forces"
device: cpu
batch_size: 10
max_num_epochs: 20
swa: True
seed: 2

Overwriting config/config-02.yml


In [8]:
train_mace("/config/config-01.yml")
#train_mace("config/config-02.yml")

usage: program [-h] --name NAME [--seed SEED] [--log_dir LOG_DIR]
               [--model_dir MODEL_DIR] [--checkpoints_dir CHECKPOINTS_DIR]
               [--results_dir RESULTS_DIR] [--downloads_dir DOWNLOADS_DIR]
               [--device {cpu,cuda,mps}] [--default_dtype {float32,float64}]
               [--log_level LOG_LEVEL]
               [--error_table {PerAtomRMSE,TotalRMSE,PerAtomRMSEstressvirials,PerAtomMAE,TotalMAE,DipoleRMSE,DipoleMAE,EnergyDipoleRMSE}]
               [--model {BOTNet,MACE,ScaleShiftMACE,ScaleShiftBOTNet,AtomicDipolesMACE,EnergyDipolesMACE}]
               [--r_max R_MAX] [--radial_type {bessel,gaussian}]
               [--num_radial_basis NUM_RADIAL_BASIS]
               [--num_cutoff_basis NUM_CUTOFF_BASIS]
               [--interaction {RealAgnosticResidualInteractionBlock,RealAgnosticAttResidualInteractionBlock,RealAgnosticInteractionBlock}]
               [--interaction_first {RealAgnosticResidualInteractionBlock,RealAgnosticInteractionBlock}]
        

SystemExit: 2